In [ ]:
from google.colab import files
uploaded = files.upload()


Saving converted_jpgs.zip to converted_jpgs.zip


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving yolo_labels.zip to yolo_labels.zip


In [ ]:
import zipfile
import os

zip_file = "/content/yolo_labels.zip"

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    extract_folder = zip_file.replace(".zip", "")
    zip_ref.extractall(extract_folder)

print(f"Extracted to: /content/{extract_folder}")


Extracted to: /content//content/yolo_labels


In [ ]:
import os
import zipfile
import shutil
import random
from pathlib import Path

base_dir = "/content"
images_zip_path = os.path.join(base_dir, "converted_jpgs.zip")
labels_dir = os.path.join(base_dir, "yolo_labels")
images_dir = os.path.join(base_dir, "images")
split_ratio = 0.8

with zipfile.ZipFile(images_zip_path, 'r') as zip_ref:
    zip_ref.extractall(images_dir)

image_files = list(Path(images_dir).glob("*.jpg"))
random.shuffle(image_files)

train_count = int(len(image_files) * split_ratio)
train_images = image_files[:train_count]
val_images = image_files[train_count:]

for split in ['train', 'val']:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'labels', split), exist_ok=True)

def move_files(image_list, split):
    for img_path in image_list:
        img_name = img_path.name
        label_name = img_path.stem + ".txt"

        shutil.move(str(img_path), os.path.join(images_dir, split, img_name))

        label_src = os.path.join(labels_dir, label_name)
        label_dst = os.path.join(base_dir, 'labels', split, label_name)
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)

move_files(train_images, 'train')
move_files(val_images, 'val')

print("Dataset organized!")

data_yaml = f"""path: {base_dir}
train: images/train
val: images/val

nc: 15
names: ['Hatchback', 'Sedan', 'SUV', 'MUV', 'Bus', 'Truck', 'Three-wheeler',
        'Two-wheeler', 'LCV', 'Mini-bus', 'Mini-truck', 'tempo-traveller',
        'bicycle', 'Van', 'Others']
"""

with open(os.path.join(base_dir, 'data.yaml'), 'w') as f:
    f.write(data_yaml)

print("data.yaml created at /content/data.yaml")


Dataset organized!
data.yaml created at /content/data.yaml


In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os
import shutil
import random
from pathlib import Path

base_dir = "/content"
images_dir = os.path.join(base_dir, "converted_jpgs")
labels_dir = os.path.join(base_dir, "yolo_labels")
split_ratio = 0.8

image_files = [p for p in Path(images_dir).glob("*.*") if p.suffix.lower() in ['.jpg', '.jpeg', '.png']]
random.shuffle(image_files)

train_count = int(len(image_files) * split_ratio)
train_images = image_files[:train_count]
val_images = image_files[train_count:]

for split in ['train', 'val']:
    os.makedirs(os.path.join(images_dir, split), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'labels', split), exist_ok=True)

def move_files(image_list, split):
    for img_path in image_list:
        img_name = img_path.name
        label_name = img_path.stem + ".txt"

        shutil.move(str(img_path), os.path.join(images_dir, split, img_name))

        label_src = os.path.join(labels_dir, label_name)
        label_dst = os.path.join(base_dir, 'labels', split, label_name)
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)

move_files(train_images, 'train')
move_files(val_images, 'val')

print("Train/Val split done!")


Train/Val split done!


In [ ]:
import os
import shutil
import random
from pathlib import Path

images_dir = "/content/images/converted_jpgs"
labels_dir = "/content/yolo_labels"
split_ratio = 0.8

image_files = [p for p in Path(images_dir).glob("*.*") if p.suffix.lower() in ['.jpg', '.jpeg', '.png']]
print(f"Found {len(image_files)} image files.")

random.shuffle(image_files)
train_count = int(len(image_files) * split_ratio)
train_images = image_files[:train_count]
val_images = image_files[train_count:]

for split in ['train', 'val']:
    os.makedirs(f"{images_dir}/{split}", exist_ok=True)
    os.makedirs(f"/content/labels/{split}", exist_ok=True)

def move_files(img_list, split):
    for img_path in img_list:
        img_name = img_path.name
        label_name = img_path.stem + ".txt"

        shutil.move(str(img_path), f"{images_dir}/{split}/{img_name}")

        src_label = os.path.join(labels_dir, label_name)
        dst_label = f"/content/labels/{split}/{label_name}"
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)

move_files(train_images, 'train')
move_files(val_images, 'val')

print("Split done.")


Found 913 image files.
Split done.


In [ ]:
!mkdir -p /content/images/converted_jpgs/train/images
!mkdir -p /content/images/converted_jpgs/val/images

!mv /content/images/converted_jpgs/train/*.jpg /content/images/converted_jpgs/train/images/
!mv /content/images/converted_jpgs/val/*.jpg /content/images/converted_jpgs/val/images/


In [ ]:
import os
import shutil

label_source = "/content/yolo_labels/yolo_labels"
base_dir = "/content/images/converted_jpgs"

for split in ["train", "val"]:
    img_dir = os.path.join(base_dir, split, "images")
    label_dir = os.path.join(base_dir, split, "labels")
    os.makedirs(label_dir, exist_ok=True)

    for img_file in os.listdir(img_dir):
        name, _ = os.path.splitext(img_file)
        src = os.path.join(label_source, name + ".txt")
        dst = os.path.join(label_dir, name + ".txt")
        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            print(f"Missing label: {name}.txt")


In [ ]:
import yaml

# Load the existing data.yaml file
with open('data.yaml', 'r') as file:
    data = yaml.safe_load(file)

# Update only the fields you want to change
data['train'] = '/content/images/converted_jpgs/train/images'
data['val'] = '/content/images/converted_jpgs/val/images'
data['nc'] = 15
data['names'] = [
    'Hatchback', 'Sedan', 'SUV', 'MUV', 'Bus', 'Truck', 'Three-wheeler',
    'Two-wheeler', 'LCV', 'Mini-bus', 'Mini-truck', 'tempo-traveller',
    'bicycle', 'Van', 'Others'
]

# Write the updated data back to the same file
with open('data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

print("Updated data.yaml successfully.")


Updated data.yaml successfully.


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
model.train(data='/content/data.yaml', epochs=150, imgsz=640, batch=16)


Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, p

train: Scanning /content/images/converted_jpgs/train/labels... 730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 730/730 [00:02<00:00, 324.32it/s]

train: /content/images/converted_jpgs/train/images/10017.jpg: 646 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10036.jpg: 169 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10060.jpg: 57 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10120.jpg: 327 duplicate labels removed
train: /content/images/converted_jpgs/train/images/1017.jpg: 565 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10216.jpg: 7 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10217.jpg: 283 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10230.jpg: 312 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10242.jpg: 423 duplicate labels removed
train: /content/images/converted_jpgs/train/images/1030.jpg: 344 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10305.jpg: 884 duplicate labels removed
trai

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 51.7±27.8 MB/s, size: 265.9 KB)


val: Scanning /content/images/converted_jpgs/val/labels... 183 images, 0 backgrounds, 0 corrupt: 100%|██████████| 183/183 [00:01<00:00, 154.88it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     83/150        12G      2.164      2.208      1.721       1834        640: 100%|██████████| 46/46 [00:18<00:00,  2.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:13<00:00,  2.23s/it]

                   all        183      24286      0.173     0.0803      0.102     0.0774



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     84/150      11.7G      2.164      2.173      1.703       3968        640:  43%|████▎     | 20/46 [00:08<00:11,  2.36it/s]

In [ ]:
from google.colab import files
files.download('runs/detect/train3/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

results = model('/content/images/converted_jpgs/val/images/1017.jpg')



image 1/1 /content/images/converted_jpgs/val/images/1017.jpg: 384x640 2 Three-wheelers, 6 Two-wheelers, 47.3ms
Speed: 2.9ms preprocess, 47.3ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:
!pip install --upgrade ultralytics

from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # You can change to yolov9s.pt, yolov9e.pt, etc.

model.train(
    data="/content/data.yaml",
    epochs=50,
    imgsz=640,
    degrees=10,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.0005,
    flipud=0.1,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4
)


100%|██████████| 21.5M/21.5M [00:00<00:00, 178MB/s]


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0005, plots=True, pose=12.0, pretrained=True, p

100%|██████████| 755k/755k [00:00<00:00, 25.1MB/s]

Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

 19                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1   1969152  ultralytics.nn.modules.block.C2f             [768, 512, 1]                 
 22        [15, 18, 21]  1   2121853  ultralytics.nn.modules.head.Detect           [15, [128, 256, 512]]         
Model summary: 129 layers, 11,141,405 parameters, 11,141,389 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

      38/50      12.3G      2.269      2.259      1.838       4790        640: 100%|██████████| 46/46 [00:23<00:00,  2.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:14<00:00,  2.35s/it]

                   all        183      23852      0.229     0.0768      0.127     0.0954



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      39/50      12.8G       2.25      2.256      1.825       6493        640:  50%|█████     | 23/46 [00:11<00:14,  1.57it/s]

WARNING ⚠️ CUDA OutOfMemoryError in TaskAlignedAssigner, using CPU


In [1]:
from ultralytics import YOLO

model = YOLO("/content/runs/detect/train/weights/last.pt")
model.train(resume=True)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=10, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.1, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=/content/runs/detect/train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0005, plots=Tru

train: Scanning /content/images/converted_jpgs/train/labels.cache... 730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 730/730 [00:00<?, ?it/s]

train: /content/images/converted_jpgs/train/images/10036.jpg: 169 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10060.jpg: 57 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10120.jpg: 327 duplicate labels removed
train: /content/images/converted_jpgs/train/images/1017.jpg: 565 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10216.jpg: 7 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10230.jpg: 312 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10242.jpg: 423 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10355.jpg: 192 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10360.jpg: 127 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10367.jpg: 956 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10392.jpg: 315 duplicate labels removed
tra

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 53.7±14.0 MB/s, size: 316.7 KB)


val: Scanning /content/images/converted_jpgs/val/labels.cache... 183 images, 0 backgrounds, 0 corrupt: 100%|██████████| 183/183 [00:00<?, ?it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 2/6 [00:06<00:13,  3.36s/it]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:18<00:00,  3.11s/it]


                   all        183      23852      0.241     0.0717      0.132      0.103
             Hatchback        175       2432      0.298       0.09      0.167      0.131
                 Sedan        168       1486      0.341     0.0922      0.203      0.167
                   SUV        173       1448      0.246     0.0684      0.136      0.113
                   MUV        160        838      0.242     0.0632      0.131      0.105
                   Bus        153        711       0.29     0.0872      0.131     0.0978
                 Truck        172       1605      0.132     0.0947     0.0791     0.0429
         Three-wheeler        177       3298      0.433     0.0855      0.235      0.185
           Two-wheeler        180       9378     0.0662      0.145      0.103     0.0612
                   LCV        172       1754      0.327     0.0633      0.159      0.114
              Mini-bus         87        152     0.0828     0.0197     0.0477     0.0396
       tempo-travelle

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b9a81138990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.0

In [ ]:
from ultralytics import YOLO

# Load your trained model (e.g., last.pt or best.pt)
model = YOLO('/content/runs/detect/train/weights/last.pt')

# Continue training for more epochs
model.train(data='data.yaml', epochs=50)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12

train: Scanning /content/images/converted_jpgs/train/labels.cache... 730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 730/730 [00:00<?, ?it/s]

train: /content/images/converted_jpgs/train/images/10036.jpg: 169 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10060.jpg: 57 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10120.jpg: 327 duplicate labels removed
train: /content/images/converted_jpgs/train/images/1017.jpg: 565 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10216.jpg: 7 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10230.jpg: 312 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10242.jpg: 423 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10355.jpg: 192 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10360.jpg: 127 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10367.jpg: 956 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10392.jpg: 315 duplicate labels removed
tra

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 614.4±111.1 MB/s, size: 316.7 KB)


val: Scanning /content/images/converted_jpgs/val/labels.cache... 183 images, 0 backgrounds, 0 corrupt: 100%|██████████| 183/183 [00:00<?, ?it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      49/50      13.8G      1.898      1.879      1.666       1483        640: 100%|██████████| 46/46 [00:17<00:00,  2.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.32s/it]

                   all        183      23852       0.22     0.0656     0.0937     0.0703



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      6.96G      1.846      1.828       1.62       1720        640:  37%|███▋      | 17/46 [00:06<00:10,  2.77it/s]

In [4]:
from ultralytics import YOLO

# Load your trained model (e.g., last.pt or best.pt)
model = YOLO('/content/runs/detect/train/weights/last.pt')

# Continue training for more epochs
model.train(data='data.yaml', epochs=5)


Ultralytics 8.3.168 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/runs/detect/train/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.

train: Scanning /content/images/converted_jpgs/train/labels.cache... 730 images, 0 backgrounds, 0 corrupt: 100%|██████████| 730/730 [00:00<?, ?it/s]

train: /content/images/converted_jpgs/train/images/10036.jpg: 169 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10060.jpg: 57 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10120.jpg: 327 duplicate labels removed
train: /content/images/converted_jpgs/train/images/1017.jpg: 565 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10216.jpg: 7 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10230.jpg: 312 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10242.jpg: 423 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10355.jpg: 192 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10360.jpg: 127 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10367.jpg: 956 duplicate labels removed
train: /content/images/converted_jpgs/train/images/10392.jpg: 315 duplicate labels removed
tra

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 630.5±45.9 MB/s, size: 316.7 KB)


val: Scanning /content/images/converted_jpgs/val/labels.cache... 183 images, 0 backgrounds, 0 corrupt: 100%|██████████| 183/183 [00:00<?, ?it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  17%|█▋        | 1/6 [00:02<00:10,  2.17s/it]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:18<00:00,  3.16s/it]


                   all        183      23852      0.202     0.0784      0.129      0.103
             Hatchback        175       2432      0.266       0.09      0.146      0.117
                 Sedan        168       1486      0.316     0.0935      0.192      0.161
                   SUV        173       1448       0.23     0.0698      0.127      0.109
                   MUV        160        838      0.178     0.0668      0.103     0.0866
                   Bus        153        711      0.158     0.0985      0.129      0.103
                 Truck        172       1605      0.123      0.102     0.0836     0.0468
         Three-wheeler        177       3298      0.398     0.0876      0.215      0.171
           Two-wheeler        180       9378      0.059      0.166      0.109     0.0695
                   LCV        172       1754       0.22     0.0775      0.127     0.0904
              Mini-bus         87        152     0.0725     0.0197     0.0498     0.0407
       tempo-travelle

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b9c03213a10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.0